
<b><font size = 2><span style="color:#4DCCBD"> 🖼Stable Diffusion with auto prompt generation 🖼  </span></font></b>  

<b><font size = 2><span style="color:#4DCCBD">Created By Burhanuddin Latsaheb </span></font> </b> 


# <center><b><font size = 6><span style="color:#F77F82">🖼Stable Diffusion with auto prompt generation 🖼 </span></font></b></center>  

## <center><font size =4><span style="color:#6BA6AA"> If you find this notebook useful,support with an upvote👍👍 </span></font></center>


In [1]:
%pip install diffusers -q
%pip install --upgrade transformers
from IPython.display import clear_output
clear_output()


 # <center><font size = 3><span style="color:#a8dadc"> <p style="background-color:#a8dadc;font-family:newtimeroman;color:#F77F82;font-size:200%;text-align:center;border-radius:100px 10px;">INTRODUCTION</p>   </span></font></center>
<font size = 4><span style="color:#6BA6AA">The major problem in creating an image using stable diffusion is thinking of the appropriate prompt, so in this notebook I have used gpt2 to generate  appropriate prompts for generating a high resolution image using stable diffusion 2.1 model.
 
<font size = 5><span style="color:#4DF77F82CCBD">Notebook Overview : </span></font>

* <font size = 3><span style="color:#3A3E59"> This notebook contains:  </span></font>
    1. <font size = 3><span style="color:#3A3E59"> Using a pretrained GPT2 model for prompt generation </span></font>
    2. <font size = 3><span style = "color:#3A3E59">Using a pretrained pipeline for stable diffusion </span></font>
*  <font size = 3><span style="color:#3A3E59">The pretrained gpt2 model for prompt generation is available on <a href =https://huggingface.co/Gustavosta/MagicPrompt-Stable-Diffusion>promt-generation-model</a> </span></font>
*  <font size = 3><span style="color:#3A3E59">The stable diffusion pipelines and schedulers are available on <a href=https://huggingface.co/spaces/stabilityai/stable-diffusion>hugging-face</a></span></font>



<a id = "top"></a>
# <center><font size = 3><span style="color:#a8dadc"> <p style="background-color:#a8dadc;font-family:newtimeroman;color:#F77F82;font-size:200%;text-align:center;border-radius:100px 10px;">TABLE OF CONTENTS</p>   </span></font></center>

- [1. Imports](#imports)
- [2. Helper Functions](#helper_functions)
- [3.Prompt Generation model](#gpt2)
- [4. Stable Diffusion Model Pipeline](#stable)
- [5. long range view, Beautiful Japanese flower garden](#1)
- [6. photo of a ultra realistic sailing ship](#2)
- [7. a magical world in a giant tea cup](#3)
- [8. beautiful, young woman, cybernetic](#4)
- [9. panting of a stylish woman with a sword](#5)
 



<a id = "imports"></a>
# <center><font size = 3><span style="color:#a8dadc"> <p style="background-color:#a8dadc;font-family:newtimeroman;color:#F77F82;font-size:200%;text-align:center;border-radius:100px 10px;">1. IMPORTS</p>   </span></font></center>    

#### [Top ↑](#top)

In [2]:
import gc
import tensorflow as tf
import torch
from PIL import Image
import IPython.display 
from torch import autocast

from tqdm.auto import tqdm

from transformers import GPT2Tokenizer, GPT2LMHeadModel , pipeline
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from transformers import pipeline, set_seed

2023-05-02 17:56:57.733445: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/veronicarhody/Desktop/pma/vnstock/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



<a id = "helper_functions"></a>
# <center><font size = 3><span style="color:#a8dadc"> <p style="background-color:#a8dadc;font-family:newtimeroman;color:#F77F82;font-size:200%;text-align:center;border-radius:100px 10px;">2. HELPER FUNCTIONS</p>   </span></font></center>    
#### [Top ↑](#top)

In [3]:
def generate_prompts(prompt , nlp , num_return_sequences = 6):
    outs = nlp(prompt, max_length=80, num_return_sequences=num_return_sequences)

    print('\nInput:\n' + 100 * '-')
    print('\033[96m' + prompt + '\033[0m')
    print('\nOutput:\n' + 100 * '-')
    for i in range(len(outs)):
        outs[i] = str(outs[i]['generated_text']).replace('  ', '')
    print('\033[92m' + '\n\n'.join(outs) + '\033[0m\n')
    return outs


In [4]:
def image_grid(imgs , rows , cols):
    w,h = imgs[0].size
    grid = Image.new("RGB" , size = (cols*w , rows*h))
    grid_w , grid_h = grid.size
    for i , img in enumerate(imgs):
        grid.paste(img , box = (i%cols*w , i//cols*h))
    return grid

In [5]:
def generate_image(prompt , pipe,num):
    print('\033[96m' + prompt + '\033[0m')
    print('\nOutput:\n' + 100 * '-')
    image = pipe(prompt).images[0]
    image.save(f"{num}.jpg")
    return image


<a id = "gpt2"></a>

# <center><font size = 3><span style="color:#a8dadc"> <p style="background-color:#a8dadc;font-family:newtimeroman;color:#F77F82;font-size:200%;text-align:center;border-radius:100px 10px;">3.PROMPT GENERATION MODEL</p>   </span></font></center>    
#### [Top ↑](#top)

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = GPT2LMHeadModel.from_pretrained('Gustavosta/MagicPrompt-Stable-Diffusion', pad_token_id=tokenizer.eos_token_id)
nlp = pipeline('text-generation', model=model, tokenizer=tokenizer)


ConnectionError: HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Read timed out.

In [ ]:
print('\033[96m' +'Prompt Generation model pipeline created !!'+ '\033[0m')

Prompt Generation model pipeline created !!



<a id = "stable"></a>

# <center><font size = 3><span style="color:#a8dadc"> <p style="background-color:#a8dadc;font-family:newtimeroman;color:#F77F82;font-size:200%;text-align:center;border-radius:100px 10px;">4. STABLE DIFFUSION MODEL PIPELINE</p>   </span></font></center>    
#### [Top ↑](#top)

In [ ]:

model_id = "stabilityai/stable-diffusion-2-1"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_attention_slicing()
pipe = pipe.to("cuda")

NameError: name 'StableDiffusionPipeline' is not defined

In [ ]:
print('\033[96m' +'Stable Diffusion model pipeline created !!'+ '\033[0m')


<a id = "1"></a>
<div style="background-color:#E8EFEC; text-align:center; vertical-align: middle; padding:40px 0;">
<b><font size = 3><span style="color:#4DCCBD">  long range view, Beautiful Japanese flower garden</span></font></b>
</div>

#### [Top ↑](#top)

In [ ]:
prompt = "long range view, Beautiful Japanese flower garden"
outs = generate_prompts(prompt , nlp , 6)

In [ ]:
imgs = []
img1 = generate_image(outs[0] , pipe , 1)
imgs.append(img1)
img1

In [ ]:
img2 = generate_image(outs[1] , pipe , 2)
imgs.append(img2)
img2

In [ ]:
img3 = generate_image(outs[2] , pipe , 3)
imgs.append(img3)
img3

In [ ]:
img4 = generate_image(outs[3] , pipe , 4)
imgs.append(img4)
img4

In [ ]:
img5 = generate_image(outs[4] , pipe , 5)
imgs.append(img5)
img5

In [ ]:
img6 = generate_image(outs[5] , pipe , 6)
imgs.append(img6)
img6

In [ ]:
image_grid(imgs, 3,2)

<a id = "2"></a>
<div style="background-color:#E8EFEC; text-align:center; vertical-align: middle; padding:40px 0;">
<b><font size = 3><span style="color:#4DCCBD">photo of a ultra realistic sailing ship</span></font></b>
</div>

#### [Top ↑](#top)

In [ ]:
prompt = "photo of a ultra realistic sailing ship"
outs = generate_prompts(prompt , nlp , 6)

In [ ]:
imgs = []
img1 = generate_image(outs[0] , pipe , 7)
imgs.append(img1)
img1

In [ ]:
img2 = generate_image(outs[1] , pipe , 8)
imgs.append(img2)
img2

In [ ]:
img3 = generate_image(outs[2] , pipe , 9)
imgs.append(img3)
img3

In [ ]:
img4 = generate_image(outs[3] , pipe , 10)
imgs.append(img4)
img4

In [ ]:
img5 = generate_image(outs[4] , pipe , 11)
imgs.append(img5)
img5

In [ ]:
img6 = generate_image(outs[5] , pipe , 12)
imgs.append(img6)
img6

In [ ]:
image_grid(imgs, 3,2)

<a id = "3"></a>
<div style="background-color:#E8EFEC; text-align:center; vertical-align: middle; padding:40px 0;">
<b><font size = 3><span style="color:#4DCCBD">a magical world in a giant tea cup</span></font></b>
</div>

#### [Top ↑](#top)

In [ ]:
prompt = "a magical world in a giant tea cup"
outs  = generate_prompts(prompt , nlp , 6)

In [ ]:
imgs = []
img1 = generate_image(outs[0] , pipe , 13)
imgs.append(img1)
img1

In [ ]:
img2 = generate_image(outs[1] , pipe , 14)
imgs.append(img2)
img2

In [ ]:
img3 = generate_image(outs[2] , pipe , 15)
imgs.append(img3)
img3

In [ ]:
img4 = generate_image(outs[3] , pipe , 16)
imgs.append(img4)
img4

In [ ]:
img5 = generate_image(outs[4] , pipe , 17)
imgs.append(img5)
img5

In [ ]:
img6 = generate_image(outs[4] , pipe , 18)
imgs.append(img6)
img6

In [ ]:
image_grid(imgs, 3,2)

<a id = "4"></a>
<div style="background-color:#E8EFEC; text-align:center; vertical-align: middle; padding:40px 0;">
<b><font size = 3><span style="color:#4DCCBD">beautiful, young woman, cybernetic</span></font></b>
</div>

#### [Top ↑](#top)

In [ ]:
prompt = "beautiful, young woman, cybernetic"
outs = generate_prompts(prompt , nlp , 6)

In [ ]:
imgs = []
img1 = generate_image(outs[0] , pipe , 19)
imgs.append(img1)
img1

In [ ]:
img2 = generate_image(outs[1] , pipe , 20)
imgs.append(img2)
img2

In [ ]:
img3 = generate_image(outs[2] , pipe , 21)
imgs.append(img3)
img3

In [ ]:
img4 = generate_image(outs[3] , pipe , 22)
imgs.append(img4)
img4

In [ ]:
img5 = generate_image(outs[4] , pipe , 23)
imgs.append(img5)

img5

In [ ]:
img6 = generate_image(outs[5] , pipe , 24)
imgs.append(img6)
img6

In [ ]:
image_grid(imgs, 3,2)

<a id = "5"></a>
<div style="background-color:#E8EFEC; text-align:center; vertical-align: middle; padding:40px 0;">
<b><font size = 3><span style="color:#4DCCBD">Synthwave halloween formula 1 car racing</span></font></b>
</div>

#### [Top ↑](#top)

In [ ]:
prompt = "An astronaut in a "
outs = generate_prompts(prompt , nlp , 6)

In [ ]:
imgs = []
img1 = generate_image(outs[0] , pipe , 25)
imgs.append(img1)
img1

In [ ]:
img2 = generate_image(outs[1] , pipe , 26)
imgs.append(img2)
img2

In [ ]:
img3 = generate_image(outs[2] , pipe , 27)
imgs.append(img3)
img3

In [ ]:
img4 = generate_image(outs[3] , pipe , 28)
imgs.append(img4)
img4

In [ ]:
img5 = generate_image(outs[4] , pipe , 29)
imgs.append(img5)
img5

In [ ]:
img6 = generate_image(outs[5] , pipe , 30)
imgs.append(img6)
img6

In [ ]:
image_grid(imgs, 3,2)

<center><b><font size = 3><span style="color:#2F4F4F"> Thank You for reading 😊</span></font></b></center>
<center><b><font size = 3><span style="color:#2F4F4F"> If you have any suggestions or feeback, please let me know</span></font></b></center>

<center><b><font size = 3><span style="color:#2F4F4F"> Do experiment on this  by forking this notebook and playing with your own prompts</span></font></b></center>



